In [2]:
import pandas as pd
import numpy as np
import plotly.graph_objs as go
import plotly.express as px
from dash import Dash, dcc, html, Input, Output
import matplotlib.pyplot as plt
import seaborn as sns

# Load dataset
file_path = r'D:\Data Science\Dataset\data_penjualan.csv'
df = pd.read_csv(file_path)

# Tangani missing values
df = df.dropna()

# Inisialisasi aplikasi Dash
app = Dash(__name__)

# Layout dashboard
app.layout = html.Div([
    html.H1("Dashboard Analisis Penjualan"),

    html.Div([
        # Grafik Jumlah Produk Terjual per Produk (bar)
        html.Div([
            dcc.Graph(id='grafik-produk-terjual')
        ], className='six columns'),

        # Grafik Jumlah Produk Terjual per Metode Pembayaran (pie)
        html.Div([
            dcc.Graph(id='grafik-pembayaran-terjual')
        ], className='six columns'),
    ], className='row'),

    html.Div([
        # Grafik Jumlah Produk Terjual per Lokasi Pelanggan (pie)
        html.Div([
            dcc.Graph(id='grafik-lokasi-terjual')
        ], className='six columns'),

        # Grafik Tren Musiman (line chart)
        html.Div([
            dcc.Graph(id='grafik-tren-musiman')
        ], className='six columns'),
    ], className='row')
])

# Callback untuk update grafik produk terjual
@app.callback(
    Output('grafik-produk-terjual', 'figure'),
    Input('grafik-produk-terjual', 'id')
)
def update_grafik_produk_terjual(_):
    colors = px.colors.qualitative.Vivid  # Gunakan skema warna default dari Plotly Express
    produk_count = df.groupby('Produk')['Order_ID'].count()
    fig = px.bar(produk_count, x=produk_count.index, y=produk_count.values, 
                 color=produk_count.index, color_discrete_sequence=colors,
                 labels={'x': 'Produk', 'y': 'Jumlah Produk Terjual', 'color': 'Produk'},
                 title='Jumlah Produk Terjual per Produk')
    fig.update_xaxes(tickangle=45)
    return fig

# Callback untuk update grafik pembayaran terjual
@app.callback(
    Output('grafik-pembayaran-terjual', 'figure'),
    Input('grafik-pembayaran-terjual', 'id')
)
def update_grafik_pembayaran_terjual(_):
    colors = px.colors.qualitative.Vivid  # Gunakan skema warna default dari Plotly Express
    payment_count = df.groupby('Metode_Pembayaran')['Order_ID'].count()
    fig = px.pie(payment_count, values=payment_count.values, names=payment_count.index,
                 color_discrete_sequence=colors, labels={'names': 'Metode Pembayaran', 'values': 'Jumlah Produk Terjual'},
                 title='Jumlah Produk Terjual per Metode Pembayaran')
    return fig

# Callback untuk update grafik lokasi terjual
@app.callback(
    Output('grafik-lokasi-terjual', 'figure'),
    Input('grafik-lokasi-terjual', 'id')
)
def update_grafik_lokasi_terjual(_):
    colors = px.colors.qualitative.Vivid  # Gunakan skema warna default dari Plotly Express
    location_count = df.groupby('Lokasi_Pelanggan')['Order_ID'].count()
    fig = px.pie(location_count, values=location_count.values, names=location_count.index,
                 color_discrete_sequence=colors, labels={'names': 'Lokasi Pelanggan', 'values': 'Jumlah Produk Terjual'},
                 title='Jumlah Produk Terjual per Lokasi Pelanggan')
    return fig

# Callback untuk update grafik tren musiman
@app.callback(
    Output('grafik-tren-musiman', 'figure'),
    Input('grafik-tren-musiman', 'id')
)
def update_grafik_tren_musiman(_):
    # Konversi kolom 'Tanggal_Pembelian' menjadi format datetime
    df['Tanggal_Pembelian'] = pd.to_datetime(df['Tanggal_Pembelian'])
    # Ekstrak bulan dari tanggal pembelian
    df['Bulan'] = df['Tanggal_Pembelian'].dt.month
    # Agregasi jumlah produk terjual per bulan
    monthly_count = df.groupby('Bulan')['Order_ID'].count()
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=monthly_count.index, y=monthly_count.values, mode='lines+markers',
                             marker=dict(size=10, color='skyblue', line=dict(width=2, color='DarkSlateGray')),
                             line=dict(color='skyblue', width=2),
                             connectgaps=True))
    fig.update_layout(title='Jumlah Produk Terjual Bulanan',
                      xaxis_title='Bulan', yaxis_title='Jumlah Produk Terjual',
                      xaxis=dict(tickmode='linear'))
    return fig

# Jalankan aplikasi di port yang berbeda
if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False, port=8060)